# SQL II: Grouping and Joining

_September 21, 2020_

Agenda:
- Practice with Grouping in SQLite
- Practice with different types of Joins in SQLite

<img src = "https://media.giphy.com/media/3oKIPnAiaMCws8nOsE/giphy.gif">

In [3]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np

## Part I. Grouping
Just like pandas, Sql support different types of grouping statements for performing aggregate functions and allow us to calculate various statistics for data.

Syntax:
`SELECT (*) FROM tables GROUP BY column`

In [4]:
# instantiate some data
auto = pd.read_csv('auto-mpg.csv')
tips = sns.load_dataset('tips')
small_quiz = pd.DataFrame({"students":["Raizel","Mitch","Eric"],
                          "projects":[1,2,1],
                          "grades":np.random.randint(80,100,3)})
small_grades = pd.DataFrame({"students":["Mike W","Mitch","Mike R"],
                            "quiz_score":np.random.randint(0,10,3)})

In [5]:
conn = sqlite3.connect('aggregate.db') 
c = conn.cursor()

In [17]:
# # insert these datasets into our db
# # auto.to_sql('cars', con=conn)
# tips.to_sql('gratitude', con=conn)
# #small_grades.to_sql('small_grade', con=conn)
# small_quiz.to_sql('small_quiz',con=conn)

In [18]:
# for the tips df/table, get the total number of man and woman
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [19]:
pd_query_1= tips.groupby(["sex"]).size()
pd_query_1

sex
Male      157
Female     87
dtype: int64

In [20]:
tips.sex.value_counts()

Male      157
Female     87
Name: sex, dtype: int64

Syntax: `SELECT ___ FROM ___ GROUP BY ___` 

In [21]:
# in sql 
query = """SELECT sex, COUNT(sex) FROM tips GROUP BY sex"""
c.execute(query).fetchall()

[('Female', 87), ('Male', 157)]

In [22]:
# using pandas, select the average amount tipped for time
tips.groupby(["time"])["tip"].mean()

time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [23]:
# use sql to recreate this query
query = """SELECT time, AVG(tip) FROM tips GROUP BY time"""
c.execute(query).fetchall()

[('Dinner', 3.102670454545454), ('Lunch', 2.7280882352941176)]

We can also use `GROUP BY` with certain conditions, in conjunction with filtering and ordering.

You can use :

`SELECT ___ FROM ___ WHERE ___ GROUP BY ___ `

`SELECT ___ FROM ___ GROUP BY ___ HAVING ____ `

In [10]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [13]:
# use pandas, get the average total bills for a party larger than 2 for every size of party
tips.groupby(tips["size" > 2]).groupby(["size"]).total_bill.mean()
# using pandas

TypeError: '>' not supported between instances of 'str' and 'int'

In [35]:
# use sql to recreate this query
query = """SELECT size, AVG(total_bill) FROM tips WHERE size > 2 GROUP BY size"""
c.execute(query).fetchall()

[(3, 23.27763157894737), (4, 28.61351351351351), (5, 30.068), (6, 34.83)]

In [36]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [50]:
# your turn - use pandas, get the maximum value of total bill for female non smoker

tips[(tips.sex == "Female")].groupby(["smoker"]).total_bill.max()["No"]
# using group by?

35.83

In [47]:
# how to get this without using group by?
tips[(tips.sex == "Female") & (tips.smoker == "No")].total_bill.max()

35.83

In [53]:
# use sql to recreate this query
query = """SELECT total_bill FROM tips WHERE smoker = "No" AND sex = "Female" ORDER BY total_bill DESC LIMIT 1"""

c.execute(query).fetchall()

[(35.83,)]

In [ ]:
# your turn - use pandas, get the average tip values for different size groups, and show top 2
# most amount of average tips


In [ ]:
# use sql 
query = None
c.execute(query).fetchall()

## Part II. Join

Based on this diagram, how would you explain the different types of joins?

<img src = 'sql-joins.png' width = 400>

In [54]:
print(small_grades)
print(small_quiz)

  students  quiz_score
0   Mike W           3
1    Mitch           1
2   Mike R           4
  students  projects  grades
0   Raizel         1      89
1    Mitch         2      91
2     Eric         1      92


In [55]:
# left join - join small quiz with small grades, on student name
small_grades.merge(small_quiz, left_on = 'students', right_on = 'students', how = "left")

# if i run this command, what would happen? how many observations/record?

,students,quiz_score,projects,grades
0,Mike W,3,NaN,NaN
1,Mitch,1,2.0,91.0
2,Mike R,4,NaN,NaN


In [1]:
# sql 
query = """SELECT * 
        FROM small_grades 
        LEFT JOIN small_quiz on small_grade.students == small_quiz.students """
c.execute(query).fetchall()

NameError: name 'c' is not defined

In [ ]:
# inner join and get the student's grades
small_grades.merge(small_quiz, on = 'students')

In [59]:
query = """SELECT * FROM small_quiz LEFT JOIN small_grade ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()


[(0, 'Raizel', 1, 89, None, None, None),
 (1, 'Mitch', 2, 91, 1, 'Mitch', 1),
 (2, 'Eric', 1, 92, None, None, None)]

In [ ]:
# your turn - use right join and get all entries

# pandas

In [ ]:
# sql 

### Level up 
1. Subqueries (including anti-joins)
2. Get median (using OFFSET)

In [ ]:
# anti-join :get all the entries that are in the left table but not in the right table 
query = """SELECT left_table.id FROM `left table` WHERE left_table.id not in (SELECT id FROM right_table) """

In [ ]:
# get median - use offset 
query = """SELECT col FROM table ORDER BY col LIMIT 1 OFFSET (SELECT COUNT(*) FROM MyTable) / 2"""

In [24]:
list = [19  18, 21, 16, 15, 17, 20, 18]

In [29]:
sum(list)/len(list)
    
    


18.0

In [34]:
(-17 + 4)/2


-6.5